<a href="https://colab.research.google.com/github/pranavkantgaur/training_materials/blob/master/intro_to_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Slide 1: Problem Specification - HighwayEnv Test Cases
Test Case 1: Lane Merging
*"Safely merge into dense traffic (1.5 vehicles/lane) while maintaining speed >18m/s"*

Test Case 2: Emergency Braking
*"Avoid collision when lead vehicle suddenly brakes at 8m/s² from 30m distance"*

Test Case 3: Multi-Lane Navigation
*"Navigate to 4th lane in heavy traffic (0.8 vehicles/lane) within 100 timesteps"*

Success Metrics:

Collision Rate (%)

Average Speed (m/s)

Lane Change Success

Slide 2: Rule-Based Solutions
Merging Test:

python
if front_vehicle_distance < 20m:
    change_lane()
else:
    accelerate()
Pros: Interpretable, Low compute
Cons: Fails in edge cases

Emergency Braking:
Fixed threshold braking (15m)
Limitation: No anticipation

Multi-Lane:
Sequential lane changes
Issue: No traffic adaptation

Slide 3: Bayesian Optimization
Focus Test Case: Emergency Braking

python
optimize(brake_threshold, speed_threshold)
Advantage: Data-efficient tuning
Drawback: No state memory

Result:
Collision Rate ↓ 40% → 15%

Slide 4: Deep RL (DQN/PPO)
Architecture:
Observation → Neural Network → Action

Test Case Performance:

Metric	DQN	PPO
Merging Speed	22.1m/s	24.3m/s
Braking Coll.	10%	5%
Tradeoff:

Adapts to complex scenarios

Requires significant training

Slide 5: Transformer Agent
Key Innovation:
Self-attention for vehicle interactions

Multi-Lane Results:

python
Attention_weights = f(relative_positions)
Advantage: Handles 10+ vehicles
Challenge: Needs large dataset

Slide 6: Classical Methods Relevance
A Path Planning:*

Global route planning ✔️

Real-time driving ❌

MCTS:

Used in Waymo's simulation ✔️

Too slow for 100km/h ❌

Conclusion:
Hybrid (Classical + RL) approaches win

Slide 7: Performance Comparison

Method	Merging	Braking	Multi-Lane
Rule-Based	18.2m/s	40% CR	60% Success
Bayesian	-	15% CR	-
DQN	22.1m/s	10% CR	75% Success
Transformer	24.3m/s	5% CR	90% Success
Slide 8: Hands-On Session Guide

Edit rule_based_agent() for TC2

Tune Bayesian parameters

Modify DQN reward function:

python
reward += (new_lane == target_lane) * 10
Visualize attention weights

Slide 9: Ethical Considerations
Merging Dilemma:
"Should the agent cut off another vehicle to avoid slowdown?"

Technical Debt:
Simulation vs real-world gap

Slide 10: Q&A
Discussion Topics:

When to choose RL over rules?

How validate safety critical systems?

Transformer vs LSTM for driving?



In [ ]:
!pip install gymnasium highway-env stable-baselines3 scikit-optimize torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import gymnasium
from gymnasium.wrappers import RecordVideo
import highway_env
from stable_baselines3 import DQN
import os

# Training
env = gymnasium.make("highway-v0")
model = DQN('MlpPolicy', env,
            policy_kwargs=dict(net_arch=[256, 256]),
            learning_rate=5e-4,
            buffer_size=15000,
            learning_starts=200,
            batch_size=32,
            gamma=0.8,
            train_freq=1,
            gradient_steps=1,
            target_update_interval=50,
            verbose=1,
            tensorboard_log="highway_dqn/")
model.learn(int(2e2))
model.save("highway_dqn/model")

# Testing with Video
env = gymnasium.make("highway-v0", render_mode='rgb_array')
env = RecordVideo(env, "highway_dqn/videos", episode_trigger=lambda x: True)
os.makedirs("highway_dqn/videos", exist_ok=True)

obs, info = env.reset()
done = truncated = False
while not (done or truncated):
    action, _ = model.predict(obs, deterministic=True)
    obs, _, done, truncated, _ = env.step(action)
env.close()

# Show video
from IPython.display import HTML
from base64 import b64encode

video_path = "highway_dqn/videos/rl-video-episode-0.mp4"
mp4 = open(video_path, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""<video width=400 controls><source src="{data_url}"></video>""")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to highway_dqn/DQN_1
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 8.25     |
|    ep_rew_mean      | 6.14     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 2        |
|    time_elapsed     | 13       |
|    total_timesteps  | 33       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 14.9     |
|    ep_rew_mean      | 11.2     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 2        |
|    time_elapsed     | 48       |
|    total_timesteps  | 119      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 15.2     |
|

/usr/local/lib/python3.11/dist-packages/moviepy/config_defaults.py:1: DeprecationWarning: invalid escape sequence '\P'
  """


In [ ]:
import gymnasium
from gymnasium.wrappers import RecordVideo
import highway_env
import numpy as np
import os

def rule_based_policy(obs):
    ego_speed = obs[0, 3]
    front_vehicle = obs[1:][np.argwhere(obs[1:, 1] == obs[0, 1])]
    if front_vehicle.size > 0 and front_vehicle[0, 0] < 20:
        if obs[0, 1] > 0: return 0  # Lane left
        else: return 4  # Slow down
    return 3 if ego_speed < 25 else 1  # Accelerate

# Testing
env = gymnasium.make("highway-v0", render_mode='rgb_array')
env = RecordVideo(env, "rule_based/videos", episode_trigger=lambda x: True)
os.makedirs("rule_based/videos", exist_ok=True)

obs, info = env.reset()
done = truncated = False
while not (done or truncated):
    action = rule_based_policy(obs)
    obs, _, done, truncated, _ = env.step(action)
env.close()

# Show video
from IPython.display import HTML
from base64 import b64encode

video_path = "rule_based/videos/rl-video-episode-0.mp4"
mp4 = open(video_path, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""<video width=400 controls><source src="{data_url}"></video>""")

In [ ]:
import gymnasium
from gymnasium.wrappers import RecordVideo
import highway_env
from skopt import gp_minimize
from skopt.space import Real, Integer
import numpy as np
import os

space = [Integer(0, 2), Real(10, 30)]  # Action mode, safety distance

def evaluate(params):
    env = gymnasium.make("highway-v0")
    total_reward = 0
    obs, _ = env.reset()
    done = truncated = False
    while not (done or truncated):
        action = 3 if obs[0, 3] < 25 else 1  # Simple speed control
        if params[0] == 0 and obs[0, 1] > 0: action = 0
        elif params[0] == 1: action = 2
        obs, reward, done, truncated, _ = env.step(action)
        total_reward += reward
    return -total_reward

result = gp_minimize(evaluate, space, n_calls=2, random_state=0)
best_params = result.x

# Test best params
env = gymnasium.make("highway-v0", render_mode='rgb_array')
env = RecordVideo(env, "bo_agent/videos", episode_trigger=lambda x: True)
os.makedirs("bo_agent/videos", exist_ok=True)

obs, _ = env.reset()
done = truncated = False
while not (done or truncated):
    action = 3 if obs[0, 3] < 25 else 1
    if best_params[0] == 0 and obs[0, 1] > 0: action = 0
    elif best_params[0] == 1: action = 2
    obs, _, done, truncated, _ = env.step(action)
env.close()

# Show video
from IPython.display import HTML
from base64 import b64encode

video_path = "bo_agent/videos/rl-video-episode-0.mp4"
mp4 = open(video_path, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""<video width=400 controls><source src="{data_url}"></video>""")

In [ ]:
import gymnasium
from gymnasium.wrappers import RecordVideo
import highway_env
from stable_baselines3 import PPO
import os

# Training
env = gymnasium.make("highway-v0")
model = PPO('MlpPolicy', env,
            policy_kwargs=dict(net_arch=[256, 256]),
            learning_rate=3e-4,
            n_steps=512,
            batch_size=64,
            n_epochs=10,
            gamma=0.9,
            verbose=1)
model.learn(int(2e1))
model.save("highway_ppo/model")

# Testing
env = gymnasium.make("highway-v0", render_mode='rgb_array')
env = RecordVideo(env, "highway_ppo/videos", episode_trigger=lambda x: True)
os.makedirs("highway_ppo/videos", exist_ok=True)

obs, info = env.reset()
done = truncated = False
while not (done or truncated):
    action, _ = model.predict(obs, deterministic=True)
    obs, _, done, truncated, _ = env.step(action)
env.close()

# Show video
from IPython.display import HTML
from base64 import b64encode

video_path = "highway_ppo/videos/rl-video-episode-0.mp4"
mp4 = open(video_path, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""<video width=400 controls><source src="{data_url}"></video>""")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 15.4     |
|    ep_rew_mean     | 11.6     |
| time/              |          |
|    fps             | 2        |
|    iterations      | 1        |
|    time_elapsed    | 200      |
|    total_timesteps | 512      |
---------------------------------


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/save_util.py:284: UserWarning: Path 'highway_ppo' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [ ]:
import gymnasium
from gymnasium.wrappers import RecordVideo
import highway_env
from stable_baselines3 import PPO
from torch import nn
import torch
import os

class TransformerPolicy(nn.Module):
    def __init__(self, feature_dim, action_dim):
        super().__init__()
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=64, nhead=4),
            num_layers=2
        )
        self.fc = nn.Linear(feature_dim, 64)
        self.head = nn.Linear(64, action_dim)

    def forward(self, x):
        x = self.fc(x)
        x = self.transformer(x.unsqueeze(0)).squeeze(0)
        return self.head(x)

# Training
env = gymnasium.make("highway-v0")
policy_kwargs = dict(
    features_extractor_class=TransformerPolicy,
    features_extractor_kwargs=dict(feature_dim=25, action_dim=5)
)
model = PPO('MlpPolicy', env,
            policy_kwargs=policy_kwargs,
            learning_rate=3e-4,
            verbose=1)
model.learn(int(2e1))
model.save("transformer_agent/model")

# Testing
env = gymnasium.make("highway-v0", render_mode='rgb_array')
env = RecordVideo(env, "transformer_agent/videos", episode_trigger=lambda x: True)
os.makedirs("transformer_agent/videos", exist_ok=True)

obs, info = env.reset()
done = truncated = False
while not (done or truncated):
    action, _ = model.predict(obs, deterministic=True)
    obs, _, done, truncated, _ = env.step(action)
env.close()

# Show video
from IPython.display import HTML
from base64 import b64encode

video_path = "transformer_agent/videos/rl-video-episode-0.mp4"
mp4 = open(video_path, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""<video width=400 controls><source src="{data_url}"></video>""")

## OLD

In [ ]:
!pip install gymnasium>=0.29.1 highway-env stable-baselines3 pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [ ]:
import gymnasium
import highway_env
from stable_baselines3 import DQN

env = gymnasium.make("highway-v0")
model = DQN('MlpPolicy', env,
              policy_kwargs=dict(net_arch=[256, 256]),
              learning_rate=5e-4,
              buffer_size=15000,
              learning_starts=200,
              batch_size=32,
              gamma=0.8,
              train_freq=1,
              gradient_steps=1,
              target_update_interval=50,
              verbose=1,
              tensorboard_log="highway_dqn/")
model.learn(int(2e1))
model.save("highway_dqn/model")

# Load and test saved model
model = DQN.load("highway_dqn/model")
while True:
  done = truncated = False
  obs, info = env.reset()
  while not (done or truncated):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = env.step(action)
    env.render()


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to highway_dqn/DQN_3


/usr/local/lib/python3.11/dist-packages/highway_env/envs/common/abstract.py:292: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("highway-v0", render_mode="rgb_array")
  gym.logger.warn(
/usr/local/lib/python3.11/dist-packages/highway_env/envs/common/abstract.py:292: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("highway-v0", render_mode="rgb_array")
  gym.logger.warn(
/usr/local/lib/python3.11/dist-packages/highway_env/envs/common/abstract.py:292: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("highway-v0", render_mode="rgb_array")
  gym.logger.warn(
/usr/local/lib/python3.11/dist-packages/highway_env/envs/common/abstract.py:292: UserWarning: WARN: You are calling render method wit

KeyboardInterrupt: 

In [ ]:
TODO: https://github.com/Farama-Foundation/HighwayEnv/blob/master/scripts/